In [81]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import librosa
import librosa.display
import os
import tensorflow as tf

In [82]:
#music_genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
data_dir="../input/gtzan-spectogram-5/images_original"
music_genres = ['blues', 'classical', 'jazz' , 'metal', 'pop']
batch_size = 50
img_height = 288
img_width = 432

In [83]:
import keras

In [84]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image_dataset_from_directory


In [85]:
training_set = image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode='categorical',
  color_mode='rgb',
  shuffle=True,
)

Found 499 files belonging to 5 classes.
Using 350 files for training.


In [86]:
testing_set = image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode='categorical',
  color_mode='rgb',
  shuffle=True,
)

Found 499 files belonging to 5 classes.
Using 149 files for validation.


In [87]:
AUTOTUNE = tf.data.AUTOTUNE

training_set = training_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
testing_set = testing_set.cache().prefetch(buffer_size=AUTOTUNE)

In [88]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from keras.layers import Dropout, BatchNormalization
from keras.layers.experimental.preprocessing import Rescaling

In [89]:
num_classes = 5

model=Sequential()

# model.add(Rescaling(1./255, input_shape=(img_height, img_width, 3)))
          
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax')) 

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [90]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 286, 430, 16)      448       
_________________________________________________________________
batch_normalization_23 (Batc (None, 286, 430, 16)      64        
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 143, 215, 16)      0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 143, 215, 16)      0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 141, 213, 32)      4640      
_________________________________________________________________
batch_normalization_24 (Batc (None, 141, 213, 32)      128       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 70, 106, 32)      

In [91]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            )
callbacks = [earlystop, learning_rate_reduction]

In [92]:
model.fit_generator(
    generator=training_set,
    validation_data=testing_set,
    verbose=1,
    callbacks=callbacks,
    epochs=120,
)

Epoch 1/120
7/7 [==============================] - 4s 286ms/step - loss: 50.6518 - accuracy: 0.2194 - val_loss: 53.9248 - val_accuracy: 0.3087
Epoch 2/120
7/7 [==============================] - 1s 111ms/step - loss: 27.0872 - accuracy: 0.3798 - val_loss: 63.1394 - val_accuracy: 0.1678
Epoch 3/120
7/7 [==============================] - 1s 111ms/step - loss: 14.4378 - accuracy: 0.5715 - val_loss: 24.4822 - val_accuracy: 0.3154
Epoch 4/120
7/7 [==============================] - 1s 111ms/step - loss: 6.9318 - accuracy: 0.7251 - val_loss: 41.0167 - val_accuracy: 0.2819
Epoch 5/120
7/7 [==============================] - 1s 111ms/step - loss: 4.4859 - accuracy: 0.6689 - val_loss: 14.5802 - val_accuracy: 0.3490
Epoch 6/120
7/7 [==============================] - 1s 110ms/step - loss: 2.3068 - accuracy: 0.8121 - val_loss: 6.9807 - val_accuracy: 0.4899
Epoch 7/120
7/7 [==============================] - 1s 111ms/step - loss: 0.7705 - accuracy: 0.9135 - val_loss: 4.8782 - val_accuracy: 0.6107
Epoch

In [94]:
model.save('./MSD_CNN_5.h5')